In [ ]:
!pip install names

In [1]:
import nltk
from nltk.corpus import stopwords, names
from nltk.corpus import names as nm
import string
import names
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
import pandas as pd
import pickle
import numpy as np
import csv


# Import Data and Split into Training and Test Sets

In [18]:
horror_scripts_raw = pickle.load(open('hor_scripts.p','rb'))

In [19]:
romance_scripts_raw = pickle.load(open('rom_scripts.p','rb'))

In [20]:
fantasy_scripts_raw = pickle.load(open('fan_scripts.p','rb'))

In [23]:
def dict_shuffle(dict_):
    l = list(dict_.items())
    random.shuffle(l)
    dict_1 = dict(l)
    return dict_1

In [24]:
horror_scripts_raw_shuff = dict_shuffle(horror_scripts_raw)

In [25]:
romance_scripts_raw_shuff = dict_shuffle(romance_scripts_raw)

In [26]:
fantasy_scripts_raw_shuff = dict_shuffle(fantasy_scripts_raw)

In [27]:
# label = {'Romance':0,'Horror':2,'Fantasy':3}

In [51]:
len(horror_scripts_raw_shuff.keys()), len(romance_scripts_raw_shuff.keys()), len(fantasy_scripts_raw_shuff.keys())

(147, 197, 116)

In [52]:
len(horror_scripts_raw_shuff['28 DAYS LATER'])

110566

In [58]:
keys_list_train = []
keys_list_test = []
X_train = []
X_test = []
y_train = []
y_test = []

In [59]:
def train_test_split(data, feat_train, label_train, feat_test, label_test, label):
    counter = 0
    for k,v in data.items():
        if counter < 92:
            print('train',k,counter);
            feat_train.append(v)
            keys_list_train.append(k)
            label_train.append(label)
            counter +=1
        elif counter >= 92 and counter < 116:
            print('test',k,counter);
            feat_test.append(v)
            keys_list_test.append(k)
            label_test.append(label)
            counter +=1
        else:
            break

In [60]:
train_test_split(romance_scripts_raw_shuff,X_train,y_train,X_test,y_test,0)
train_test_split(horror_scripts_raw_shuff,X_train,y_train,X_test,y_test,1)
train_test_split(fantasy_scripts_raw_shuff,X_train,y_train,X_test,y_test,2)

train THE LAST OF THE MOHICANS 0
train "WHITE CHRISTMAS" 1
train THE BIJOU 2
train Crouching Tiger, Hidden Dragon 3
train MARLEY & ME 4
train Chasing Amy 5
train "Never Been Kissed" 6
train THE ARTIST 7
train THE GREAT GATSBY 8
train AIRPLANE 9
train HER 10
train JUNO 11
train "IT'S A WONDERFUL LIFE" 12
train T H E   A M E R I C A N   P R E S I D E N T 13
train MRS BROWN BY JEREMY BROCK 14
train W I L D  A T  H E A R T 15
train "BIRTHDAY GIRL" 16
train PRIDE AND PREJUDICE 17
train HIGH FIDELITY 18
train VANILLA SKY 19
train PEARL HARBOR - by Randall Wallace 20
train SILVER LININGS PLAYBOOK 21
train "The Princess Bride" 22
train 500 DAYS OF SUMMER 23
train CHARADE 24
train THE THEORY OF EVERYTHING 25
train EASY "A" 26
train DATE NIGHT 27
train "THE HUDSUCKER PROXY" 28
train TALL IN THE SADDLE 29
train FADE IN 30
train M  U  L  H  O  L  L  A  N  D 31
train THE BODYGUARD 32
train HEAVENLY CREATURES 33
train AUTUMN IN NEW YORK 34
train THE BATTLE OF SHAKER HEIGHTS 35
train A WALK TO REMEMB

In [43]:
# keys_list_test = []
# for k,v in horror_scripts_raw.items():
#     if k not in keys_list_train:
#         print(k)
#         test_horror.append(v)
#         keys_list_test.append(k)

In [63]:
len(X_train), len(y_train),len(keys_list_train)

(276, 276, 276)

In [64]:
len(X_test), len(y_test), len(keys_list_test)

(72, 72, 72)

In [65]:
X_train[239][0:40]

'SHREK Written by William Steig & Ted Ell'

# Stopwords and Tokenize

In [66]:
stopwords_list = stopwords.words("english")
stopwords_list += string.punctuation
stopwords_list += ["''", '""', '...', '`']
stopwords_list += [str(num) for num in range(0,10)]
stopwords_list += list(map(lambda x: x.lower(), nm.words()))

In [ ]:
# def process_script(script):
#     pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
#     script_tokens_raw =  nltk.regexp_tokenize(script, pattern)
#     script_tokens = [word.lower() for word in script_tokens_raw]
#     script_words_stopped = [word for word in script_tokens if word not in stopwords_list]
#     return script_words_stopped

##### With Lemme

In [67]:
lemmatizer = WordNetLemmatizer()
def process_script_w_lemme(script):
    pattern = "([a-zA-Z]+(?:'[a-z]*)?)"
    script_tokens_raw =  nltk.regexp_tokenize(script, pattern)
    script_tokens = [word.lower() for word in script_tokens_raw]
    script_words_stopped = [lemmatizer.lemmatize(word) for word in script_tokens if word not in stopwords_list]
    script_words_stopped = [lemmatizer.lemmatize(word,pos='a') for word in script_words_stopped if word not in stopwords_list]
    script_words_stopped = [lemmatizer.lemmatize(word,pos='v') for word in script_words_stopped if word not in stopwords_list]
    return script_words_stopped

In [ ]:
# processed_script_train = list(map(process_script, X_train))
# processed_script_test = list(map(process_script, X_test))

In [ ]:
# pickle.dump(processed_script_train,open('processed_script_train.p','wb'))

In [ ]:
# pickle.dump(processed_script_test,open('processed_script_test.p','wb'))

In [ ]:
# len(processed_script_train[0])

In [68]:
processed_script_train_lemme = list(map(process_script_w_lemme, X_train))
processed_script_test_lemme = list(map(process_script_w_lemme, X_test))

In [69]:
pickle.dump(processed_script_train_lemme,open('processed_script_train_lemme.p','wb'))

In [70]:
pickle.dump(processed_script_test_lemme,open('processed_script_test_lemme.p','wb'))

In [71]:
len(processed_script_train_lemme[0])

13351

In [72]:
len(processed_script_train), processed_script_train[2][0:3]

NameError: name 'processed_script_train' is not defined

# FreqDist

In [ ]:
# def total_vocab(list_s):
#    total_vocab = set()
#     for list_ in list_s:
#         total_vocab.update(list_)
        

In [ ]:
# processed_scripts_freqdist_train = list(map(FreqDist, processed_script_train))

In [ ]:
# processed_scripts_freqdist_train[0]

##### With Lemme

In [73]:
processed_scripts_freqdist_train_lemme = list(map(FreqDist, processed_script_train_lemme))

In [74]:
processed_scripts_freqdist_train_lemme[0]

FreqDist({'hawkeye': 348, 'heyward': 238, 'uncas': 153, 'magua': 140, 'huron': 134, 'chingachgook': 130, 'munro': 126, 'look': 101, 'cut': 90, 'hand': 88, ...})

In [ ]:
# Testing data

In [75]:
processed_scripts_freqdist_test_lemme = list(map(FreqDist, processed_script_test_lemme))

# Create Data Frame

In [ ]:
# train_df = pd.DataFrame(processed_scripts_freqdist_train[0], index=[0])

In [ ]:
# train_df

In [ ]:
# train_df = pd.DataFrame(processed_scripts_freqdist_train[0], index=[0])
# for i in range(1,len(processed_scripts_freqdist_train)):
#     try:
#         right = pd.DataFrame(processed_scripts_freqdist_train[i], index=[i])
#         print(right.head(1), right.shape)
#         train_df = pd.concat([train_df, right],  ignore_index=True)
#         print('success', keys_list_train[i])
#     except:
#         print('fail', keys_list_train[i])
#         continue
        

In [ ]:
# df.groupby(df.columns, axis=1).agg(numpy.sum)

In [ ]:
# train_df.fillna(value=0,axis=0, inplace=True)

In [ ]:
# train_df.shape

In [ ]:
# train_df.iloc[100:103,10000:10100]

In [ ]:
# full_train_df = train_df.copy()
# full_train_df.insert(0, "Movie_name", keys_list_train, False)
# full_train_df.insert(1, "Genre_label", y_train, False)

In [ ]:
# full_train_df.head(10)

##### With Lemme

In [76]:
train_df_lemme = pd.DataFrame(processed_scripts_freqdist_train_lemme[0], index=[0])
for i in range(1,len(processed_scripts_freqdist_train_lemme)):
    try:
        right_lemme = pd.DataFrame(processed_scripts_freqdist_train_lemme[i], index=[i])
        print(right_lemme.head(1), right_lemme.shape)
        train_df_lemme = pd.concat([train_df_lemme, right_lemme],  ignore_index=True)
        print('success', keys_list_train[i])
    except:
        print('fail', keys_list_train[i])
        continue

   white  christmas  krasna  panama  full  shoot  rear  area  day  glass  ...  \
1      7         24       1       1    31    149     5    10   25      6  ...   

   clearly  immense  steer  beneath  sprig  onstage  completely  extreme  \
1        1        1      1        1      1        1           1        1   

   miniature  frame  
1          1      1  

[1 rows x 1840 columns] (1, 1840)


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


success "WHITE CHRISTMAS"
   bijou  revise  draft  october  magic  around  time  everywhere  every  \
2     60       1      1        2      7      42    55           3     16   

   thing  ...  happily  relative  term  sale  envelop  tempo  bijou's  neon  \
2     31  ...        1         1     1     1        1      1        1     1   

   chaser  illuminate  
2       1           1  

[1 rows x 2679 columns] (1, 2679)
success THE BIJOU
   crouch  tiger  hide  dragon  hui  ling  schamus  tsai  kuo  jung  ...  \
3       1      2    11       4    1     1        1    35    1     1  ...   

   whatever  climb  reach  magic  pillow  gorge  shroud  remain  roll  cheek  
3         1      1      1      1       1      1       1       1     1      1  

[1 rows x 1566 columns] (1, 1566)
success Crouching Tiger, Hidden Dragon
   write  roos  march  black  microphone  feedback  follow  v  okay   go  ...  \
4      8     1      1      4           2         1      26  8    65  108  ...   

   mine  gras

success 500 DAYS OF SUMMER
    charade  write  stone  base  story  behm  october  st  fade  title  ...  \
24        4      7      4     2      6     1        1  11     4      3  ...   

    film  mancini  mercer  pose  masquerade  fate  sad  serenade  heart's  \
24     1        1       1     1           1     1    1         1        1   

    compose  
24        1  

[1 rows x 2701 columns] (1, 2701)
success CHARADE
    theory  everything  write  mccarten  shoot  script  november  predict  \
25     105         108     19         1      9       1         1        3   

    wrong  hawk  ...  implode  liquid  n  nebula  cosmic  sell  million  \
25      6    44  ...        1       1  1       1       1     1        1   

    worldwide  grandchild  credit  
25          1           1       1  

[1 rows x 2383 columns] (1, 2383)
success THE THEORY OF EVERYTHING
    easy  write   v  first  draft  darkness  rumor  promiscuity  greatly  \
26     3      4  33     20      1         2      9        

success MARTY
    clueless
44         1 (1, 1)
success Clueless
    natural  bear  killer  write  tarantino  int  coffee  shop  day  \
45        5     6      15     14          1   74      13    12  121   

    somewhere  ...  bond  passport  music  colorada  puppet  climb  \
45          3  ...     1         2      1         1       1      1   

    peacefully  eventually  noja  production  
45           1           1     1           1  

[1 rows x 2610 columns] (1, 2610)
success "NATURAL BORN KILLERS"
    yes  man  write  stoller  base  book  first  draft  february  ext  ...  \
46  125   38     11        1     1    12     14      1         1   32  ...   

    minivan  rundown  buzzer  boisterous  delight  bend  majesty  \
46        1        1       1           1        1     1        1   

    masturbatory  feast  omar's  
46             1      1       1  

[1 rows x 2194 columns] (1, 2194)
success YES MAN
    cool  write  kramer  ext  stygian  darkness  night  suggestion  travel  \
4

...

success PIRATES OF THE CARRIBEAN: DEAD MAN'S CHEST
     field  dream  write  march  final  draft  screenplay  fade  montage  \
185    100     19     20      2      1      1           1     7        3   

     photo  ...  could've  square  heaven's  silver  wisely  hint  moisture  \
185      7  ...         1       1         1       1       1     1         1   

     forth  bath  satiny  
185      1     1       1  

[1 rows x 2296 columns] (1, 2296)
success ?
     batman  screenplay  hamm  base  character  create  first  draft  october  \
186     163           1     1     4          5       4     16      1        1   

     fade  ...  void  triple  moonless  sparkle  settle  brief  skyward  sear  \
186     4  ...     1       1         1        1       1      1        1     1   

     dominate  adventure  
186         1          1  

[1 rows x 3830 columns] (1, 3830)
success BATMAN
     perfect  creature  write  standring  shoot  script  ext  prologue  black  \
187       15        30     

success THE ARMY OF DARKNESS
     dogma  ext  asbury  boardwalk  day  jersey  spring  beyond  wooden  \
207      3   19       2          5   50      25       1       3       1   

     plank  ...  prize  carousel  puke  thrash  bend  pucker  pinch  ust  \
207      1  ...      1         1     1       1     1       1      1    1   

     arri  stall  
207     1      1  

[1 rows x 3423 columns] (1, 3423)
success DOGMA
     fantastic  four  frost  base  marvel  comic  book  shoot  script  fade  \
208         20    28      1     3       1      1     3     13       1     7   

     ...  showoff  harbor  wooden  operation  cell  leonard's  cellphone  \
208  ...        1       1       1          1     1          1          1   

     destination  stencil  freighter  
208            1        1          1  

[1 rows x 2766 columns] (1, 2766)
success FANTASTIC FOUR
     tmnt  write  create  eastman  first  draft  polish  fade  ext  fill  ...  \
209     9      1       4        1     14      1    

success THE EMPIRE STRIKES BACK
     little  mermaid  complete  script  compile  johanningmeier  portion  \
227      26        7         2       1        1               1        1   

     copyright  c  disney  ...  impale  dy  horribly  collapse  revert  lead  \
227          1  1       1  ...       1   1         1         1       1     1   

     return  hug  finale  sail  
227       1    1       1     1  

[1 rows x 1238 columns] (1, 1238)
success THE LITTLE MERMAID
     script  burton's  nightmare  christmas  narrator  twas  long  time  ago  \
228       1         1          2         26         1     1     6    28    1   

     seem  ...  screen  wolfman  f  wife  mind  gaze  sit  plain  zoom  heaven  
228     6  ...       2        1  1     1     1     1    1      1     1       1  

[1 rows x 1071 columns] (1, 1071)
success The script to
     ghost  screenplay  shoot  draft  int  tribeca  loft  day  camera  track  \
229     52           1     26      1   73        2    41   60     

success BOOK OF THE DEAD
     hellraiser  iii  hell  earth  write  atkins  original  story  randel  st  \
248           5    1    17      5      4       2         1     15       1   1   

     ...  scour  zap  heel  weapon  nuance  respect  world's  summerskill'  \
248  ...      1    2     1       1       1        1        1             1   

     office  sophistication  
248       2               1  

[1 rows x 2527 columns] (1, 2527)
success HELLRAISER III: HELL ON EARTH
     viking  b  window  top  location  href
249       1  4       1    2         2     2 (1, 6)
success Erik the Viking
     fade  ext  suburbia  night  beautiful  late  summer  cricket  chirp  \
250     1   68         1     31          8    20       1        1      1   

     sprinkler  ...  singe  lady's  absolute  spill  seed  multiply  beware  \
250          1  ...      1       1         1      1     1         1       1   

     progeny  unholy  union  
250        1       1      1  

[1 rows x 2057 columns] (1, 20

success LEGEND
     barbarian  write  donnelly  oppenheimer  base  e  october  black  \
268         12      3         1            1     4  1        1     23   

     darkness  hear  ...  gust  decimate  debris  carefully  stain  threaten  \
268        16    18  ...     1         1       1          1      1         1   

     grim  funeral  pyre  extinguish  
268     1        1     2           1  

[1 rows x 2352 columns] (1, 2352)
success CONAN THE BARBARIAN
     complex  write  richards  fade  close  shoot  quinn'  leg  nervous  \
269        3      1         1     3     12      8      13    5        4   

     shake  ...  stem  ma'am  likewise  handshake  eat  lovebird  dusted'  \
269      4  ...     1      1         2          1    1         1        1   

     final  content  dust  
269      1        1     1  

[1 rows x 1352 columns] (1, 1352)
success CHRIST COMPLEX
     movie  mewtwo  strike  back  official  manuscript  translation  \
270      3     114       1    16         1   

In [77]:
train_df_lemme.shape

(276, 49799)

In [78]:
train_df_lemme.fillna(value=0,axis=0, inplace=True)

In [79]:
full_train_df_lemme = train_df_lemme.copy()
full_train_df_lemme.insert(0, "Movie_name", keys_list_train, False)
full_train_df_lemme.insert(1, "Genre_label", y_train, False)

In [80]:
full_train_df_lemme.shape

(276, 49801)

In [81]:
pickle.dump(full_train_df_lemme,open('full_train_df_lemme.p','wb'))

In [82]:
full_train_df_lemme_filtered = full_train_df_lemme.copy()

In [83]:
for word in full_train_df_lemme_filtered.columns[2:]:
    counts = full_train_df_lemme_filtered[word].value_counts().reset_index()
    if counts[counts['index']!=0.0][word].sum()<2:
        full_train_df_lemme_filtered.drop(word,axis=1,inplace=True)
full_train_df_lemme_filtered.head()

,Movie_name,Genre_label,a',a'commin,a'course,a'la,a'mighty,a's,aa,aaa,...,zowie,zpeak,zurich,zut,zuzu,zuzu',zuzu's,zy,zzzzt,zzzzzzt
0,THE LAST OF THE MOHICANS,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"""WHITE CHRISTMAS""",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,THE BIJOU,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Crouching Tiger, Hidden Dragon",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MARLEY & ME,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
# pickle.dump(full_train_df_lemme_filtered,open('full_train_df_lemme_filtered_2+.p','wb'))

Dropping words that only appear in a handful of scripts. They are not going to be good features since they will have very few overlap with other movies.

|Number of movies |  Words left |
|-----------------|-------------|
|2+               |   24624     |
|3+               |   12435     |
|4+               |   11209     |
|5+               |   10251     |
|6+               |   9483      |
|7+               |   8823      |
|10+              |   8308      |
|15+              |   6476      |

In [85]:
new_df = pd.DataFrame(index=full_train_df_lemme.index)
new_df['Movie_name'] = full_train_df_lemme['Movie_name']
new_df['Genre_label'] = full_train_df_lemme['Genre_label']
for index, row in full_train_df_lemme.iterrows():
    words = row[2:].sort_values(ascending=False)[:25].index.tolist()
    for word in words:
        new_df[word] = full_train_df_lemme[word]
new_df
# new_df = pd.DataFrame(index=full_train_df_lemme_filtered['Movie_name'])
# new_df.append(full_train_df_lemme_filtered['Genre_label'])
# new_df.head()

,Movie_name,Genre_label,hawkeye,heyward,uncas,magua,huron,chingachgook,munro,look,...,excalibur,mordred,truman's,monitor,hushpuppy,wink,bathtub,hushpuppy's,daddy,walrus
0,THE LAST OF THE MOHICANS,0,348.0,238.0,153.0,140.0,134.0,130.0,126.0,101.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"""WHITE CHRISTMAS""",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,THE BIJOU,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0
3,"Crouching Tiger, Hidden Dragon",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MARLEY & ME,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,188.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0
5,Chasing Amy,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,142.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"""Never Been Kissed""",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,THE ARTIST,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,THE GREAT GATSBY,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
9,AIRPLANE,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,...,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0


In [86]:
new_df.shape

(276, 1273)

In [101]:
new_df.index=new_df['Movie_name']
labels = new_df['Genre_label']
features = new_df.drop(columns=['Genre_label','Movie_name'])

In [102]:
# pickle.dump(new_df,open('new_df.p','wb'))

In [ ]:
# test dataframe

In [ ]:
processed_scripts_freqdist_test_lemme[0]

In [88]:
test_df = pd.DataFrame(processed_scripts_freqdist_test_lemme[0], index=[0])
for i in range(1,len(processed_scripts_freqdist_test_lemme)):
    try:
        right = pd.DataFrame(processed_scripts_freqdist_test_lemme[i], index=[i])
        print(right.head(1), right.shape)
        test_df = pd.concat([test_df, right],  ignore_index=True)
        print('success', keys_list_test[i])
    except:
        print('fail', keys_list_test[i])
        continue

   forever  write  rashida  jones  mccormack  montage  open  credit  levine'  \
1        9      5        1      1          1        5    11       1        1   

   rhino  ...  awhile  gas  station  gum  water  blatantly  gallon  restrain  \
1      3  ...       1    3        2    1      2          1       1         1   

   dog's  thirsty  
1      1        1  

[1 rows x 2168 columns] (1, 2168)


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


success CELESTE AND JESSE FOREVER
   script  b  window  top  location  href  there's  something  decter  j  \
2       1  4       1    2         2     2        1          1       1  1   

   strauss  farrelly  
2        1         2   (1, 12)
success <script>
   ultimo  tango  parigi  last  paris  write  bertolucci  franco  arcalli  \
3       1     10       1     7      8      3           1       1        1   

   final  ...  roof  curl  fetal  position  corpse  alibi  rape  lunatic  \
3      1  ...     1     1      1         1       1      1     2        2   

   swell  credit  
3      1       1  

[1 rows x 1778 columns] (1, 1778)
success ULTIMO TANGO A PARIGI
   n  g  h  l  screenplay  title  ext  various  day  she'  ...  leicester  \
4  2  2  1  3           1      1   39        2  101     1  ...          1   

   square  ultimately  green  child  papoose  tai  chi  group  mysteriously  
4       1           1      1      1        1    1    1      1             1  

[1 rows x 1914 colu

success THE BACK-UP PLAN
    punch  drink  anderson  b  break  frame  script  website  abstraction  \
22      1      1         1  8      1      1       2        1            1   

    http  ...  free  notice  must  stay  entact  use  window  top  location  \
22     1  ...     1       1     1     1       1    1       1    2         2   

    href  
22     2  

[1 rows x 22 columns] (1, 22)
success PUNCH-DRUNK LOVE by Paul Thomas Anderson
    get  marry  write  lumet  ext  halfway  house  porch  day  darkly  ...  \
23   81      5      2      1   37        4     52     18   60       1  ...   

    maroon  camry  idle  vaporize  chilly  visible  carseat  wiggle  ghost  \
23       1      1     1         1       1        1        1       1      1   

    luminous  
23         1  

[1 rows x 2468 columns] (1, 2468)
success RACHEL GETTING MARRIED
    ghost  ship  formerly  chimera  hanlon  first  draft  fade  int  barge  \
24      2    85         1      214       1     16      2     3  170    

    haunt  house  self  revision  tolkin  base  novel  initial  shoot  script  \
43      4    108     3         1       1     1      1        2     11       2   

    ...  suggest  mountain  printer  proudly  shout  dedicate  they'll  \
43  ...        1         1        4        1      1         1        1   

    immortal  whiten  today  
43         1       1      1  

[1 rows x 2632 columns] (1, 2632)
success THE HAUNTING OF HILL HOUSE
    thing  screenplay  lancaster  base  story   go  w  campbell  jr  second  \
44     47           1          1     1      1  114  1         1   1       7   

    ...  frostbitten  guardedly  suspiciously  non  chessboard  fire's  \
44  ...            1          1             1    1           1       1   

    neither  painfully  healthy  dance  
44        1          1        1      1  

[1 rows x 2434 columns] (1, 2434)
success "THE THING"
    hellraiser  iii  hell  earth  write  atkins  original  story  randel  st  \
45           5    1    17      5 

success THE CHRONICLES OF NARNIA
    mask  write  verheiden  final  draft  ext  high  sea  day  dragonhead  \
63   305      1          1      4      1   54    19    3   21           2   

    ...  smack  horribly  yah  caliber  soldier  grip  ttears  spectacle  pas  \
63  ...      1         1    1        1        1     1       1          1    1   

    squirm  
63       1  

[1 rows x 3109 columns] (1, 3109)
success THE MASK
    l  b  r  n  h  phillips  jones  story  early  movie  ...  starry  \
64  1  2  1  6  1         1      1      1      2      1  ...       2   

    lovingly  focus  woozily  railway  determinedly  bedspread  journey  earn  \
64         1      2        1        1             1          1        1     1   

    gently  
64       1  

[1 rows x 2013 columns] (1, 2013)
success =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
    consideration  best  adapt  screenplay  boyens  base  book  j  r  tolkien  \
65              1     3      1           1

In [110]:
test_df.fillna(value=0,axis=0, inplace=True)

In [111]:
# pickle.dump(test_df,open('test_df.p','wb'))

# TF-IDF

In [103]:
vectorizer = TfidfVectorizer()

In [104]:
tf_idf_data_train = vectorizer.fit_transform(new_df)

In [105]:
tf_idf_data_train.shape

(1273, 1236)

In [112]:
tf_idf_data_test = vectorizer.transform(test_df)

In [113]:
tf_idf_data_test.shape

(25083, 1236)

# Modeling Time!!!!!!!!!!!

In [3]:
# new_df=pickle.load(open('new_df.p','rb'))

In [10]:
# test_df = pickle.load(open('test_df.p','rb'))

In [107]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [114]:
forest_vanilla = RandomForestClassifier()
forest_vanilla.fit(features, y_train)

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [119]:
test_df[features.columns.tolist()]

KeyError: '[\'ml\', \'spaulding\', "frankie\'s", \'jabba\', \'dupont\', \'ashe\', "franklin\'", \'lampert\', \'uther\', "guido\'s", \'ming\', \'johner\', \'bprd\', \'tron\', \'stobrod\', \'hennessey\', \'tomoko\', \'flemmer\', \'airk\', \'pintel\', \'marke\', \'necropolis\', \'yamamura\', \'trillian\', \'kael\', \'beni\', \'mercutio\', \'gump\', \'jafar\', \'blum\', \'vizzini\', "franklin\'s", \'raziel\', \'sorsha\', "truman\'s", \'pok\', \'ponsonby\', \'coffey\', "juno\'s", \'crepsley\', \'isoyg\', \'carrigan\', \'pikachu\', \'hendricks\', \'pugsley\', \'galadriel\', \'supergirl\', \'bingley\', \'kkinsa\', \'yu\', \'raph\', \'brodski\', \'cassius\', \'sebastion\', \'lassiter\', \'gumb\', \'perseus\', \'fandral\', \'daggett\', \'gediman\', \'dumont\', "ginger\'s", \'zorg\', "frank\'ll", "hushpuppy\'s", \'helvetica\', \'peshet\', \'volstagg\', \'lando\', \'derm\', \'neverending\', \'hawkmen\', \'hallstrom\', \'pagination\', "nancy\'", \'massey\', "geoff\'s", \'batman\', \'eigerman\', \'

In [120]:
y_hat_train_v_rf = forest_vanilla.predict(features)
# y_hat_test_v_rf = forest_vanilla.predict(test_df[features.columns.tolist()])
accuracy_score(y_train, y_hat_train_v_rf), #accuracy_score(y_test, y_hat_test_v_rf)

(0.9239130434782609,)